In [8]:
import librosa
import numpy as np
import tensorflow as tf
import os

In [9]:
SAMPLE_RATE = 8000
DURATION = 1.5
N_MFCC = 40

In [10]:
MODEL_PATH = "cortana_detector.keras"

In [11]:
AUDIO_FOLDER = "../samples/trash"

In [16]:
model = tf.keras.models.load_model(MODEL_PATH)

In [17]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
    mfcc_mean = np.mean(mfcc.T, axis=0)
    return mfcc_mean

In [19]:
for file in os.listdir(AUDIO_FOLDER):
    if not file.endswith(".wav"):
        continue

    path = os.path.join(AUDIO_FOLDER, file)
    features = extract_features(path)
    features = features.reshape(1, -1)

    prediction = model.predict(features, verbose=0)[0][0]
    label = "Cortana" if prediction > 0.9 else "Not Cortana"

    print(f"{file} → Prediction: {prediction:.3f} → Classified as: {label}")

165_sample.wav → Prediction: 0.000 → Classified as: Not Cortana
195_sample.wav → Prediction: 0.753 → Classified as: Not Cortana
31_sample.wav → Prediction: 0.999 → Classified as: Cortana
33_sample.wav → Prediction: 1.000 → Classified as: Cortana
364_sample.wav → Prediction: 1.000 → Classified as: Cortana
170_sample.wav → Prediction: 0.680 → Classified as: Not Cortana
143_sample.wav → Prediction: 0.002 → Classified as: Not Cortana
388_sample.wav → Prediction: 1.000 → Classified as: Cortana
389_sample.wav → Prediction: 1.000 → Classified as: Cortana
248_sample.wav → Prediction: 0.000 → Classified as: Not Cortana
19_sample.wav → Prediction: 0.998 → Classified as: Cortana
217_sample.wav → Prediction: 1.000 → Classified as: Cortana
20_sample.wav → Prediction: 1.000 → Classified as: Cortana
244_sample.wav → Prediction: 1.000 → Classified as: Cortana
224_sample.wav → Prediction: 0.007 → Classified as: Not Cortana
22_sample.wav → Prediction: 0.989 → Classified as: Cortana
113_sample.wav → Pred

---
### Audio Record And Recognize Speaker & Wakeword
---

In [ ]:
import pyaudio

In [ ]:
py = pyaudio.PyAudio()